In [8]:
import random
import networkx as nx
import pandas as pd

In [42]:
# A function to make the simple SI step. Loop over
# all neighbors of nodes in I and transition them
# to I with probability beta.
def si(G, infected_nodes, beta, vaccinated=[]):
   new_infected = infected_nodes.copy()
   # vulnerable_neighbors_of_i = {n for infected_node in infected_nodes for n in G.neighbors(infected_node)} # Typeof = set
   # vulnerable_neighbors_of_infected = [n for infected_node in infected_nodes for n in G.neighbors(infected_node)] # Typeof = list
   vulnerable_neighbors_of_infected = []
   for infected_node in infected_nodes:
      for n in G.neighbors(infected_node):
         vulnerable_neighbors_of_infected.append(n)
   for vulnerable_node in vulnerable_neighbors_of_infected:
      if random.random() < beta and vulnerable_node not in vaccinated:
         new_infected.add(vulnerable_node)
   return new_infected

In [5]:
# Load the data
df = pd.read_csv('./network.csv/edges.csv', usecols=['source', 'target'])
dfNodes = pd.read_csv('./network.csv/nodes.csv', usecols=['name', 'SEX'])
G = nx.Graph()

sourceList = df['source'].tolist()
targetList = df['target'].tolist()
# sexList = df['sex'].tolist()
added = []
for i in range(len(sourceList)):
    G.add_edges_from([(sourceList[i], targetList[i])])
    added.append((sourceList[i], targetList[i]))

nameList = dfNodes['name'].tolist()
sexList = dfNodes['SEX'].tolist()

for i in range(len(nameList)):
    G.add_nodes_from([
    (nameList[i], {"sex": sexList[i]})
])

largest_cc = max(nx.connected_components(G), key=len)
Gcc = G.subgraph(largest_cc)

sex = nx.get_node_attributes(Gcc, "sex")
sex = list(sex.values())
print(f'Number of males: {sex.count(0)}')
print(f'Number of females: {sex.count(1)}')
print(len(Gcc.nodes))

sex_assortativity = nx.attribute_assortativity_coefficient(Gcc,'sex')
print(sex_assortativity)

Number of males: 11340
Number of females: 11471
26706
0.02855217832419135


# Infection time

## Choosing starting nodes randomly

In [37]:
# Run the model for different beta values = spreading rates
for beta in (0.05, 0.1, 0.2):
   # Run it 10 times for each beta value
   ts_to_infect = []
   for _ in range(10):
      # Pick the initial infected set
      i_nodes = set(random.sample(sorted(set(Gcc.nodes)), 1))
      # Run until we infected the entire network
      t = 0
      while (len(i_nodes) / len(Gcc.nodes)) < 0.8:
         i_nodes = si(Gcc, i_nodes, beta)
         t += 1
      ts_to_infect.append(t)
   print("Beta = %1.2f, avg steps to 80%% infection: %1.2f days." % (beta, sum(ts_to_infect) / len(ts_to_infect)))

Beta = 0.05, avg steps to 80% infection: 77.30 days.
Beta = 0.10, avg steps to 80% infection: 34.70 days.
Beta = 0.20, avg steps to 80% infection: 19.60 days.


In [45]:
# Run the model for different beta values = spreading rates
for beta in (0.05, 0.1, 0.2):
   # Run it 10 times for each beta value
   ts_to_infect = []
   for _ in range(50):
      # Pick the initial infected set
      i_nodes = set(random.sample(sorted(set(Gcc.nodes)), 1))
      # Run until we infected the entire network
      t = 0
      while (len(i_nodes) / len(Gcc.nodes)) < 0.8:
         i_nodes = si(Gcc, i_nodes, beta)
         t += 1
      ts_to_infect.append(t)
   print("Beta = %1.2f, avg steps to 80%% infection: %1.2f days." % (beta, sum(ts_to_infect) / len(ts_to_infect)))

Beta = 0.05, avg steps to 80% infection: 77.88 days.
Beta = 0.10, avg steps to 80% infection: 35.20 days.
Beta = 0.20, avg steps to 80% infection: 20.30 days.


## Choosing the node with the highest degree

In [ ]:
# Let's try to deliberately choose the hubs
# Node #800 has the highest degree, we choose that one
# Run the model for different beta values = spreading rates
for beta in (0.05, 0.1, 0.2):
   # Run it 10 times for each beta value
   ts_to_infect = []
   for _ in range(10):
      # Pick the initial infected set
      i_nodes = set([list(Gcc.nodes)[list(Gcc.nodes).index(800)]])
      # Run until we infected the entire network
      t = 0
      while (len(i_nodes) / len(Gcc.nodes)) < 0.8:
         i_nodes = si(Gcc, i_nodes, beta)
         t += 1
      ts_to_infect.append(t)
   print("Beta = %1.2f, avg steps to 80%% infection: %1.2f days." % (beta, sum(ts_to_infect) / len(ts_to_infect)))

Beta = 0.05, avg steps to 80% infection: 68.70 days.
Beta = 0.10, avg steps to 80% infection: 35.00 days.
Beta = 0.20, avg steps to 80% infection: 17.80 days.


# Let's introduce vaccination

## Vaccinate the hub, spread randomly

In [ ]:
# Let's try to vaccinate some nodes

# Run the model for different beta values = spreading rates
for beta in (0.05, 0.1, 0.2):
   # Run it 10 times for each beta value
   ts_to_infect = []
   for _ in range(10):
      # Pick the initial infected set
      i_nodes = set(random.sample(sorted(set(Gcc.nodes)), 1))
      # Run until we infected the entire network
      t = 0
      while (len(i_nodes) / len(Gcc.nodes)) < 0.8:
         i_nodes = si(Gcc, i_nodes, beta, [list(Gcc.nodes)[list(Gcc.nodes).index(800)],list(Gcc.nodes)[list(Gcc.nodes).index(289)],list(Gcc.nodes)[list(Gcc.nodes).index(0)]])
         t += 1
      ts_to_infect.append(t)
   print("Beta = %1.2f, avg steps to 80%% infection: %1.2f days." % (beta, sum(ts_to_infect) / len(ts_to_infect)))

Beta = 0.05, avg steps to 80% infection: 98.80 days.
Beta = 0.10, avg steps to 80% infection: 51.40 days.
Beta = 0.20, avg steps to 80% infection: 26.90 days.


## Vaccinate 3 males?

In [ ]:
# Let's try to vaccinate some nodes

## Vaccinate 3 females

In [ ]:
# Let's try to vaccinate some nodes

## Vaccinate 3 homosexuals

In [ ]:
# Let's try to vaccinate some nodes

## Vaccinate 3 heterosexuals

In [ ]:
# Let's try to vaccinate some nodes